In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder  

train_ex1 = train_used_car.copy()

encoder = OneHotEncoder(sparse_output = False)
train_encoded = encoder.fit_transform(train_ex1[['Brand','Model']]) 

encoded_columns = encoder.get_feature_names_out(['Brand', 'Model'])
train_encoded_df = pd.DataFrame(train_encoded, columns=encoded_columns)

train_ex1 = pd.concat([train_ex1, train_encoded_df], axis=1).drop(['Brand', 'Model'], axis=1)

display(encoded_columns)
display(train_ex1.head())

In [ ]:
test_ex1 = test_used_car.copy()

try :
    test_encoded = encoder.transform(test_ex1[['Brand', 'Model']])
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoded_columns)
    test_ex1 = pd.concat([test_ex1, test_encoded_df], axis=1).drop(['Brand', 'Model'], axis=1)
    test_ex1

except Exception as e:
    print(f"Error : {e}")

```sh
Error : Found unknown categories ['Chevrolet'] in column 0 during transform
```

### 이 오류는 테스트 데이터셋의 'Brand' 컬럼에 'Chevrolet'이라는 학습 데이터셋에는 존재하지 않는 새로운 범주가 있기 때문에 발생했습니다.

- OneHotEncoder 클래스의 handle_unknown='ignore' 매개변수
- 앞서 발생한 문제를 해결하기 위해, OneHotEncoder 클래스의 handle_unknown='ignore' 옵션을 활용합니다.



In [ ]:
train_ex2 = train_used_car.copy()

encoder_ex2 = OneHotEncoder(sparse_output = False, handle_unknown ='ignore' )

train_encoded_ex2 = encoder_ex2.fit_transform(train_ex2[['Brand','Model']]) 

# Convert the encoded features into a DataFrame
encoded_columns_ex2 = encoder_ex2.get_feature_names_out(['Brand', 'Model'])
train_encoded_df2 = pd.DataFrame(train_encoded_ex2, columns=encoded_columns_ex2)

train_ex2 = pd.concat([train_ex2, train_encoded_df2], axis=1).drop(['Brand', 'Model'], axis=1)

test_ex2 = test_used_car.copy()

try :
    test_encoded_ex2 = encoder_ex2.transform(test_ex2[['Brand', 'Model']])
    test_encoded_df2 = pd.DataFrame(test_encoded_ex2, columns=encoded_columns_ex2)
    test_ex2 = pd.concat([test_ex2, test_encoded_df2], axis=1).drop(['Brand', 'Model'], axis=1)
    display(test_ex2)

except Exception as e:
    print(f"Error : {e}")

세 개의 컬럼을 모두 사용하면, 적어도 두 컬럼 간에는 일종의 연관성이 존재하게 마련입니다. 예를 들어, 'Brand_Ford'가 0이라면, 이는 자동적으로 'Brand_Honda' 또는 'Brand_Toyota' 중 하나가 1이 될 가능성이 높다는 것을 의미합니다. 이처럼 컬럼 간에 상호 의존적인 관계가 형성되는 것을 `다중공선성`이라고 합니다. 이는 모델의 예측력을 저하시키고 해석을 어렵게 만드는 주요 원인이 됩니다."

`더미 변수의 함정`
OneHotEncoder의 drop매개변수를 통해 drop='first'로 설정하면, 첫번째 카테고리를 자동으로 삭제.  
다중공선성의 위험을 줄이면서 필요한 정보를 모두 포함 시킬 수 있다. 

# 다중공선성 — 영어: multicollinearity
설명: 독립변수들 간에 강한 상관관계가 있어 회귀계수 추정이 불안정해지고 표준오차가 커지는 현상. 해결책 예: 상관 높은 변수 제거, PCA, 정규화(릿지 등).  

# 더미변수의 함정 — 영어: dummy variable trap
설명: 모든 범주에 대한 더미 변수를 포함(그리고 절편이 있을 때)하면 완전한 선형종속(완전한 다중공선성)이 발생해 계수 추정이 불가능해지는 상황. 해결책: 한 카테고리(dropping one) 제거하거나 절편을 제거.  

# 추가 자료
https://en.wikipedia.org/wiki/Multicollinearity  
https://en.wikipedia.org/wiki/Dummy_variable_trap  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html  

# drop='first'로도 다중공선성이 남는 경우
요약: OneHotEncoder(drop='first')는 각 범주형 변수 내부의 합-1(또는 절편과의 완전선형종속) 문제만 제거합니다. 서로 다른 컬럼 간에 선형적(결정적) 관계가 있거나, 엔지니어링된 특성이 더미들의 선형결합이면 drop으로는 해결되지 않습니다.

## 예시 파일
nominal_ragne_variable_encoding-1.ipynb

## 답변
간단한 예시 + 코드로 직관적으로 설명합니다. 아래 예는 Model이 Brand를 결정(네스트됨)하는 경우로, 모델 더미들의 합으로 브랜드 더미를 복원할 수 있어 drop='first'를 써도 컬럼 간 선형종속이 남습니다.

```python
# python
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Model이 Brand를 완전히 결정하는 상황(네스트됨)
df = pd.DataFrame({
    'Brand':  ['Ford','Ford','Honda','Honda'],
    'Model':  ['F1','F2','H1','H2']
})

enc = OneHotEncoder(drop='first', sparse=False)  # sklearn 버전에 따라 sparse=False / sparse_output=False 사용
X = enc.fit_transform(df[['Brand','Model']])
print("features:", enc.get_feature_names_out(['Brand','Model']))
print("design matrix:\n", X)
print("shape:", X.shape, "rank:", np.linalg.matrix_rank(X))
```

- 출력 예시 해석: design matrix의 열 수는 4개(Brand의 1개 + Model의 3개)지만 행렬 랭크(rank)는 3으로 나옵니다. 즉 열들 사이에 선형종속(다중공선성)이 존재합니다.
- 이유: 예를 들어 남아있는 브랜드 더미(Brand_Honda)는 Model_H1 + Model_H2 와 같은 방식으로 표현될 수 있으므로(drop에서 어떤 카테고리가 제거됐는지에 따라 달라짐) drop='first'로도 종속관계가 제거되지 않습니다.

추가로 drop='first'로 해결되지 않는 전형적 상황들:
- 서로 다른 범주형 컬럼 간에 결정적(혹은 강한) 관계(예: 지역→우편번호, 모델→브랜드).
- 범주형 더미 + 해당 더미의 합이나 파생 열(예: 카운트, 비율)을 동시에 포함한 경우.
- 원-핫과 레이블 인코딩(또는 ordinal) 등 서로 중복 정보를 제공하는 다른 인코딩을 함께 사용한 경우.
- 컬럼 상호작용(교호항)을 수동으로 추가했을 때 기존 더미의 선형결합으로 나타나는 경우.

해결책(간단히):
- 중복 정보를 가진 컬럼 중 하나를 제거하거나(예: Brand 또는 Model),
- Model이 Brand를 포함하는 구조면 Model만 사용하거나 Brand를 Model 기반으로 집계/병합,
- 정규화 기반 회귀(Ridge) 사용 또는 PCA로 차원 축소,
- 카테고리 관계를 반영한 피처 엔지니어링(예: 계층적 인코딩, target encoding),
- 트리 계열 모델 사용(다중공선성에 둔감).

### 추가 자료
- https://en.wikipedia.org/wiki/Multicollinearity  
- https://en.wikipedia.org/wiki/Dummy_variable_trap  
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

## binary encoding (바이너리 인코딩)

- 고유 범주의 갯수가 많은 명목형 변수를 인코딩할 수 있는 대표적인 기법 
- 데이터의 차원을 효과적으로 줄이면서 각 범주를 명확하게 표현 

|name|orinary encoding|(binary encoding)0|1|
|---|---|---|---|
|bread|1|0|1|
|milk|2|1|0|
|apple|3|1|1|

In [ ]:
import category_encoders as ce

train_ex5 = train_income.copy()

train_ex5['occupation'].fillna('Unknown', inplace=True)

# Creating a BinaryEncoder instance
encoder_ex5 = ce.BinaryEncoder(cols=['occupation'])

# Fitting and transforming the 'occupation' column
train_encoded_ex5 = encoder_ex5.fit_transform(train_ex5['occupation'])

encoded_columns_ex5 = train_encoded_ex5.columns

# Creating a DataFrame from the binary encoded data
train_encoded_df5 = pd.DataFrame(train_encoded_ex5, columns=encoded_columns_ex5)
train_ex5 = pd.concat([train_ex5, train_encoded_df5], axis=1).drop(['occupation'], axis=1)

display(encoded_columns_ex5)
display(train_ex5.head())

범주의 간소화: 데이터 탐색(EDA)과 도메인 전문 지식을 활용하여 유사한 범주를 통합하고, 빈도가 낮은 범주를 병합하는 접근법은 범주의 수를 효과적으로 줄일 수 있습니다. 이러한 방법은 단순히 모델의 복잡성을 감소시키는 것을 넘어, 인코딩 기법 자체가 미치는 영향보다 더 큰 향상을 모델 성능에 가져올 수 있습니다.

바이너리 인코딩 적용: 간소화된 범주 구조를 가진 후에는 바이너리 인코딩 방식을 적용하는 것이 권장됩니다. 바이너리 인코딩은 다수의 범주를 가진 변수에 대해 원-핫 인코딩에 비해 더 메모리 효율적이며, 이진 수 조합을 통해 각 범주를 표현함으로써 생성되는 새로운 변수의 수를 최소화합니다.

고급 인코딩 기법 고려: 본 학습에서는 다루지 않았지만, 타깃 인코딩(Target Encoding)과 같은 고급 인코딩 기법도 주목해야 합니다. 이러한 기법은 종속 변수(타깃)와의 관계를 기반으로 각 범주를 인코딩하며, 